**Load data**

In [ ]:
import pandas as pd

In [ ]:
train_data = pd.read_csv('/kaggle/input/deeptweets/train.csv')
test_data = pd.read_csv('/kaggle/input/deeptweets/test.csv')

**Explore the first lines**

In [ ]:
train_data.head()

In [ ]:
test_data.head()

**Descriptive statistics**

In [ ]:
train_data.describe()

**Information on data types**

In [ ]:
train_data.info()

****Exploratory data analysis****

Visualize the two classes in the following bar chart

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='Label', data=train_data)
plt.show()

we see that the two classes are balanced

Identify specific patterns in tweets

In [ ]:
from wordcloud import WordCloud

sport_tweets = train_data[train_data['Label'] == 'Sports']['TweetText'].values
politics_tweets = train_data[train_data['Label'] == 'Politics']['TweetText'].values

# WordCloud pour la classe "Sport"
if len(sport_tweets) > 0:
    wordcloud_sport = WordCloud(width=800, height=400, background_color='white').generate(' '.join(sport_tweets))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_sport, interpolation='bilinear')
    plt.title('WordCloud pour la Classe "Sport"')
    plt.axis('off')
    plt.show()
else:
    print("Aucun tweet dans la classe 'Sport'.")
# WordCloud pour la classe "Politique"
# Vérifier si la classe "Politique" a au moins un tweet
if len(politics_tweets) > 0:
    wordcloud_politics = WordCloud(width=800, height=400, background_color='white').generate(' '.join(politics_tweets))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_politics, interpolation='bilinear')
    plt.title('WordCloud pour la Classe "Politique"')
    plt.axis('off')
    plt.show()
else:
    print("Aucun tweet dans la classe 'Politique'.")

**Preprocessing the Data:**

> 1. Text Cleaning:

Eliminate special characters, URLs, and irrelevant symbols from the tweets.
Convert the text to lowercase to maintain consistency.
> 2. Tokenization:

Tokenize the cleaned text, breaking it into individual words (tokens).
> 3. Stopword Removal:

Remove common stopwords (e.g., "the," "and") to emphasize meaningful words.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

custom_stopwords = set(["the", "and", "is", "in", "to", "of", "it", "this", "that"])

# Function for text cleaning
def clean_text(text):
    # Remove special characters and URLs
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters and symbols
    
    # Convert the text to lowercase
    text = text.lower()
    
    return text

# Function for tokenization and stopword removal
def tokenize_and_remove_stopwords(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in custom_stopwords]
    
    return tokens

# Combine text cleaning and tokenization into a single function
def preprocess_text(text):
    cleaned_text = clean_text(text)
    tokens = tokenize_and_remove_stopwords(cleaned_text)
    return ' '.join(tokens)

# Apply preprocessing to each tweet in the dataset
train_data['ProcessedText'] = train_data['TweetText'].apply(preprocess_text)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_data['ProcessedText'], train_data['Label'], test_size=0.2, random_state=42)



In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Assuming you have 'ProcessedText' column in your datase

# WordCloud for the "Sports" class
sports_tweets = train_data[train_data['Label'] == 'Sports']['ProcessedText'].values
if len(sports_tweets) > 0:
    wordcloud_sports = WordCloud(width=800, height=400, background_color='white').generate(' '.join(sports_tweets))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_sports, interpolation='bilinear')
    plt.title('WordCloud for Class "Sports after preprocissing "')
    plt.axis('off')
    plt.show()
else:
    print("No tweets in the 'Sports' class.")

# WordCloud for the "Politics" class
politics_tweets = train_data[train_data['Label'] == 'Politics']['ProcessedText'].values
if len(politics_tweets) > 0:
    wordcloud_politics = WordCloud(width=800, height=400, background_color='white').generate(' '.join(politics_tweets))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_politics, interpolation='bilinear')
    plt.title('WordCloud for Class "Politics" after preprocissing ')
    plt.axis('off')
    plt.show()
else:
    print("No tweets in the 'Politics' class.")


In [ ]:
train_data.head()

**Feature extraction**

For feature extraction, I'll show you how to use both TF-IDF to represent tweets as numerical vectors.

 Using TF-IDF(Term Frequency-Inverse Document Frequency):

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features based on your dataset
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)


****Model Training****

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Initialize the SVM model
svm_model = SVC(kernel='linear', random_state=42)  # You can experiment with different kernels

# Train the SVM model using TF-IDF features
svm_model.fit(X_train_tfidf, y_train)

# Make predictions on the validation set
svm_predictions = svm_model.predict(X_val_tfidf)

# Evaluate the SVM model
svm_accuracy = accuracy_score(y_val, svm_predictions)
svm_classification_rep = classification_report(y_val, svm_predictions)

print(f"SVM Accuracy: {svm_accuracy:.2f}")
print("SVM Classification Report:\n", svm_classification_rep)


**Hyperparameter tuning**

Hyperparameter tuning is a crucial step in optimizing the performance of my machine learning model. For SVM, one common hyperparameter to tune is the regularization parameter **C**. The choice of the kernel (**linear**, **rbf**, **poly**, etc.) and other parameters such as the degree of the polynomial **kernel** or the **gamma** parameter can also impact the model's performance.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto']}

svm_model = SVC(random_state=42)

# Create the GridSearchCV object
random_search = RandomizedSearchCV(svm_model, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

# Perform the randomized search on the training data
random_search.fit(X_train_tfidf, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Get the best model from the randomized search
best_svm_model = random_search.best_estimator_

# Make predictions on the validation set using the best model
best_predictions = best_svm_model.predict(X_val_tfidf)

# Evaluate the best model
best_accuracy = accuracy_score(y_val, best_predictions)
best_classification_rep = classification_report(y_val, best_predictions)

print(f"Best SVM Accuracy: {best_accuracy:.2f}")
print("Best SVM Classification Report:\n", best_classification_rep)


**Performance of the model**

break down performance metrics into different stages of the process, including feature extraction, model learning and output prediction

In [ ]:
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Vectorisation des textes avec TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Ajustez max_features en fonction de votre ensemble de données
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Mesurer le temps d'exécution pour l'apprentissage du modèle SVM avec TF-IDF
start_time = time.time()
# Initialiser le modèle SVM
svm_model = SVC(kernel='linear', random_state=42)
# Entraîner le modèle SVM avec TF-IDF
svm_model.fit(X_train_tfidf, y_train)
end_time = time.time()
model_training_time = end_time - start_time

# Mesurer le temps d'exécution pour la prédiction de la sortie avec TF-IDF
start_time = time.time()
# Faire des prédictions sur l'ensemble de validation avec TF-IDF
svm_predictions = svm_model.predict(X_val_tfidf)
end_time = time.time()
output_prediction_time = end_time - start_time

# Évaluer la performance du modèle avec TF-IDF
accuracy = accuracy_score(y_val, svm_predictions)
classification_rep = classification_report(y_val, svm_predictions)

# Afficher les résultats avec les temps d'exécution
print(f"Model Training Time: {model_training_time:.4f} seconds")
print(f"Output Prediction Time: {output_prediction_time:.4f} seconds")
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_rep)


**Test the model**

To test my  model with the test data, you can follow a similar procedure to the one we used for the validation data

In [ ]:
# Définir les mêmes fonctions de prétraitement que pour les données d'entraînement
custom_stopwords = set(["the", "and", "is", "in", "to", "of", "it", "this", "that"])

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Supprimer les URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer les caractères non alphabétiques et les symboles
    text = text.lower()  # Convertir le texte en minuscules
    return text

def tokenize_and_remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in custom_stopwords]
    return tokens

def preprocess_text(text):
    cleaned_text = clean_text(text)
    tokens = tokenize_and_remove_stopwords(cleaned_text)
    return ' '.join(tokens)

# Data processing
test_data['ProcessedText'] = test_data['TweetText'].apply(preprocess_text)

X_test_tfidf = tfidf_vectorizer.transform(test_data['ProcessedText'])

svm_predictions_test = svm_model.predict(X_test_tfidf)

svm_predictions_test = svm_model.predict(X_test_tfidf)

test_data['Predicted_Label'] = svm_predictions_test
#print test Data after preprocessing
test_data.head(10)

In [ ]:
test_data.head()

In [ ]:
result_df = test_data[['TweetId', 'Predicted_Label']]
result_df.to_csv('Resultat.csv', index=False)